In [1]:
main_repo_dir = os.path.abspath(os.path.join('../..'))
sys.path.append(os.path.join(main_repo_dir, 'src'))

In [2]:
import datetime

In [3]:
import pandas

In [4]:
import utils

In [49]:
with open(os.path.join(main_repo_dir, "main_run.log"), "r") as f:
    log = f.readlines()

In [50]:
dates, entries = zip(*[(' '.join(line.split()[:3]), ' '.join(line.split()[3:])) for line in log])

In [51]:
dates = [datetime.datetime.strptime(date, '%m/%d/%Y %I:%M:%S %p') for date in dates]

In [52]:
log = pandas.DataFrame(data = {'entry' : entries}, index=dates)

In [53]:
log['day'] = [ent.strftime('%Y-%m-%d') for ent in log.index]

In [54]:
log = log['2017-03-14':]

In [55]:
log.shape

(189957, 2)

In [56]:
# Define which entry we're in
# There's got to be a better way to do this with an iterator...
entry_ind = -1

def update_entry(x):
    global entry_ind
    if x['entry'] == "Retrieving contents...":
        entry_ind += 1
    return(entry_ind)


# Flag Errors
def flag_error(x):
    text = x.entry.lower()
    if text.find('warning') > -1 or text.find('error') > -1:
        return(True)
    else:
        return(False)
    

def extract_err_class(error_text):
    start = error_text.find('<class ')
    end = error_text.find("'>:")
    if start > -1 and end > -1:
        err_class = error_text[start : (end + 2)]
    else:
        err_class = ''
    return(err_class)

In [57]:
log['entry_ind'] = log.apply(lambda x: update_entry(x), axis=1)
log['is_error'] = log.apply(lambda x: flag_error(x), axis=1)
log['err_class'] = log.entry.apply(lambda x: extract_err_class(x))

In [58]:
log.shape

(189957, 5)

In [59]:
log.head()

,entry,day,entry_ind,is_error,err_class
2017-03-14 05:00:02,Retrieving contents...,2017-03-14,0,False,
2017-03-14 05:00:02,Total feeedss to visit: 250,2017-03-14,0,False,
2017-03-14 05:00:02,Creating 8 Tor openers...,2017-03-14,0,False,
2017-03-14 05:00:04,IP Used: b'62.210.246.163',2017-03-14,0,False,
2017-03-14 05:00:06,Total entries retrieved from https://www.thegu...,2017-03-14,0,False,


In [60]:
log.tail()

,entry,day,entry_ind,is_error,err_class
2017-04-24 20:38:47,Total new links added: 0,2017-04-24,390,False,
2017-04-24 20:38:50,Total entries retrieved from https://phys.org/...,2017-04-24,390,False,
2017-04-24 20:38:51,Total new links added: 0,2017-04-24,390,False,
2017-04-24 20:38:53,Total entries retrieved from https://phys.org/...,2017-04-24,390,False,
2017-04-24 20:38:54,Total new links added: 0,2017-04-24,390,False,


## Investiate Errors

In [61]:
errors = log[log.is_error==True].copy()

In [62]:
errors.shape

(1479, 5)

In [63]:
errors.head()

,entry,day,entry_ind,is_error,err_class
2017-03-14 05:08:13,Feed http://www.ft.com/rss/world/mideast/econo...,2017-03-14,0,True,<class 'urllib.error.HTTPError'>
2017-03-14 05:09:12,Feed http://www.ft.com/rss/management/connecte...,2017-03-14,0,True,<class 'urllib.error.HTTPError'>
2017-03-14 06:08:19,Feed http://www.ft.com/rss/world/mideast/econo...,2017-03-14,1,True,<class 'urllib.error.HTTPError'>
2017-03-14 06:09:06,Feed http://www.ft.com/rss/management/connecte...,2017-03-14,1,True,<class 'urllib.error.HTTPError'>
2017-03-22 17:11:10,Feed http://feeds.foxnews.com/foxnews/latest: ...,2017-03-22,90,True,<class 'http.client.IncompleteRead'>


In [64]:
errors.tail()

,entry,day,entry_ind,is_error,err_class
2017-04-24 16:02:09,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:10,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:11,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:12,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:13,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>


In [65]:
for e in errors.err_class.unique():
    print(e)

<class 'urllib.error.HTTPError'>
<class 'http.client.IncompleteRead'>
<class 'mysql.connector.errors.DataError'>
<class 'UnboundLocalError'>
<class 'urllib.error.URLError'>
<class 'KeyError'>
<class 'mysql.connector.errors.DatabaseError'>


# DB Errors

In [66]:
ke = errors[errors.err_class=="<class 'mysql.connector.errors.DatabaseError'>"].copy()
ke.shape

(15, 5)

In [67]:
ke

,entry,day,entry_ind,is_error,err_class
2017-04-18 08:07:01,Error encountered: <class 'mysql.connector.err...,2017-04-18,328,True,<class 'mysql.connector.errors.DatabaseError'>
2017-04-18 09:06:40,Error encountered: <class 'mysql.connector.err...,2017-04-18,329,True,<class 'mysql.connector.errors.DatabaseError'>
2017-04-18 10:26:28,Error encountered: <class 'mysql.connector.err...,2017-04-18,330,True,<class 'mysql.connector.errors.DatabaseError'>
2017-04-18 11:05:03,Error encountered: <class 'mysql.connector.err...,2017-04-18,331,True,<class 'mysql.connector.errors.DatabaseError'>
2017-04-18 12:06:43,Error encountered: <class 'mysql.connector.err...,2017-04-18,332,True,<class 'mysql.connector.errors.DatabaseError'>
2017-04-18 16:07:46,Error encountered: <class 'mysql.connector.err...,2017-04-18,336,True,<class 'mysql.connector.errors.DatabaseError'>
2017-04-18 17:07:10,Error encountered: <class 'mysql.connector.err...,2017-04-18,337,True,<class 'mysql.connector.errors.DatabaseError'>
2017-04-19 07:07:36,Error encountered: <class 'mysql.connector.err...,2017-04-19,340,True,<class 'mysql.connector.errors.DatabaseError'>
2017-04-19 08:06:40,Error encountered: <class 'mysql.connector.err...,2017-04-19,341,True,<class 'mysql.connector.errors.DatabaseError'>
2017-04-20 14:07:11,Error encountered: <class 'mysql.connector.err...,2017-04-20,360,True,<class 'mysql.connector.errors.DatabaseError'>


In [68]:
ke.entry.unique()

array([ 'Error encountered: <class \'mysql.connector.errors.DatabaseError\'>: (1366, "1366 (HY000): Incorrect string value: \'\\\\xF0\\\\x9F\\\\x87\\\\xBA\\\\xF0\\\\x9F...\' for column \'summary\' at row 1", \'HY000\')',
       'Error encountered: <class \'mysql.connector.errors.DatabaseError\'>: (1366, "1366 (HY000): Incorrect string value: \'\\\\xF0\\\\x9F\\\\x94\\\\xB4 N...\' for column \'summary\' at row 1", \'HY000\')',
       'Error encountered: <class \'mysql.connector.errors.DatabaseError\'>: (1366, "1366 (HY000): Incorrect string value: \'\\\\xF0\\\\x9F\\\\x90\\\\xB8 (...\' for column \'summary\' at row 1", \'HY000\')'], dtype=object)

#### Format to different text type?

### Key Errors

In [69]:
te = errors[errors.err_class == "<class 'KeyError'>"].copy()
te.shape

(4, 5)

In [70]:
te

,entry,day,entry_ind,is_error,err_class
2017-04-07 07:04:17,Feed http://feeds.bbci.co.uk/news/video_and_au...,2017-04-07,236,True,<class 'KeyError'>
2017-04-07 07:04:28,Feed http://feeds.bbci.co.uk/news/video_and_au...,2017-04-07,236,True,<class 'KeyError'>
2017-04-24 05:07:55,Feed http://www.ft.com/rss/home/middleeast: <c...,2017-04-24,377,True,<class 'KeyError'>
2017-04-24 06:07:25,Feed http://www.ft.com/rss/home/middleeast: <c...,2017-04-24,378,True,<class 'KeyError'>


In [71]:
te.entry.unique()

array([ "Feed http://feeds.bbci.co.uk/news/video_and_audio/world/rss.xml: <class 'KeyError'>: ('link',)",
       "Feed http://feeds.bbci.co.uk/news/video_and_audio/technology/rss.xml: <class 'KeyError'>: ('link',)",
       "Feed http://www.ft.com/rss/home/middleeast: <class 'KeyError'>: ('summary',)"], dtype=object)

#### Not worth dealing with..

### MySQL Connector Errors

In [76]:
err01 = "<class 'mysql.connector.errors.DataError'>"
err02 = "<class 'mysql.connector.errors.DatabaseError'>"

#### Data Error

In [77]:
dee = errors[errors.err_class==err01]
dee.shape

(1, 5)

In [78]:
dee.tail()

,entry,day,entry_ind,is_error,err_class
2017-03-28 14:06:48,Error encountered: <class 'mysql.connector.err...,2017-03-28,139,True,<class 'mysql.connector.errors.DataError'>


Only 1, not a big deal...

In [79]:
dee.entry[-1]

'Error encountered: <class \'mysql.connector.errors.DataError\'>: (1406, "1406 (22001): Data too long for column \'link\' at row 1", \'22001\')'

#### DB Error

In [80]:
dee = errors[errors.err_class==err02]
dee.shape

(15, 5)

In [81]:
dee.tail().index

DatetimeIndex(['2017-04-24 05:07:43', '2017-04-24 06:07:17',
               '2017-04-24 08:06:09', '2017-04-24 09:07:46',
               '2017-04-24 10:05:18'],
              dtype='datetime64[ns]', freq=None)

In [82]:
len(dee.entry.unique())

3

In [83]:
dee.entry.unique()

array([ 'Error encountered: <class \'mysql.connector.errors.DatabaseError\'>: (1366, "1366 (HY000): Incorrect string value: \'\\\\xF0\\\\x9F\\\\x87\\\\xBA\\\\xF0\\\\x9F...\' for column \'summary\' at row 1", \'HY000\')',
       'Error encountered: <class \'mysql.connector.errors.DatabaseError\'>: (1366, "1366 (HY000): Incorrect string value: \'\\\\xF0\\\\x9F\\\\x94\\\\xB4 N...\' for column \'summary\' at row 1", \'HY000\')',
       'Error encountered: <class \'mysql.connector.errors.DatabaseError\'>: (1366, "1366 (HY000): Incorrect string value: \'\\\\xF0\\\\x9F\\\\x90\\\\xB8 (...\' for column \'summary\' at row 1", \'HY000\')'], dtype=object)

Strange, non-unicode string errors again.  Not worth fixing, perhaps..?

In [85]:
sec = pandas.tslib.Timedelta('1 second')

In [86]:
def get_err_contexts(errs):
    contexts = []
    for ind in errs.index:
        i = 1
        success = False
        while not success:
            sub = list(log[str(ind - i*sec)]['entry'])
            if len(sub)== 0:
                i += 1
                if i > 3:
                    success=True
            else:
                success = True
                contexts.append({'index' : str(ind),
                                 'context' :sub})
    return(contexts)

In [87]:
cons = get_err_contexts(dee)

In [88]:
cons[-5:]

[{'context': ['Total entries retrieved from http://feeds.feedburner.com/breitbart?format=xml: 46'],
  'index': '2017-04-24 05:07:43'},
 {'context': ['Total entries retrieved from http://feeds.feedburner.com/breitbart?format=xml: 46'],
  'index': '2017-04-24 06:07:17'},
 {'context': ['Total entries retrieved from http://feeds.feedburner.com/breitbart?format=xml: 47'],
  'index': '2017-04-24 08:06:09'},
 {'context': ['Total entries retrieved from http://feeds.feedburner.com/breitbart?format=xml: 47'],
  'index': '2017-04-24 09:07:46'},
 {'context': ['Total entries retrieved from http://feeds.feedburner.com/breitbart?format=xml: 47'],
  'index': '2017-04-24 10:05:18'}]

In [89]:
for i in range(-5,0):
    print(dee[cons[i]['index']]['entry'][0])

Error encountered: <class 'mysql.connector.errors.DatabaseError'>: (1366, "1366 (HY000): Incorrect string value: '\\xF0\\x9F\\x90\\xB8 (...' for column 'summary' at row 1", 'HY000')
Error encountered: <class 'mysql.connector.errors.DatabaseError'>: (1366, "1366 (HY000): Incorrect string value: '\\xF0\\x9F\\x90\\xB8 (...' for column 'summary' at row 1", 'HY000')
Error encountered: <class 'mysql.connector.errors.DatabaseError'>: (1366, "1366 (HY000): Incorrect string value: '\\xF0\\x9F\\x90\\xB8 (...' for column 'summary' at row 1", 'HY000')
Error encountered: <class 'mysql.connector.errors.DatabaseError'>: (1366, "1366 (HY000): Incorrect string value: '\\xF0\\x9F\\x90\\xB8 (...' for column 'summary' at row 1", 'HY000')
Error encountered: <class 'mysql.connector.errors.DatabaseError'>: (1366, "1366 (HY000): Incorrect string value: '\\xF0\\x9F\\x90\\xB8 (...' for column 'summary' at row 1", 'HY000')


In [94]:
feed_data = utils.load_feedlist_data('breitbart_feedlist.xml')

In [95]:
for i,f in enumerate(feed_data):
    print(str(i) + ' ' + f['Link'])

0 http://feeds.feedburner.com/breitbart?format=xml


In [99]:
rss_entry = feed_data[0]
rss_entry

{'Link': 'http://feeds.feedburner.com/breitbart?format=xml',
 'RssName': 'All News',
 'Source': 'breitbart',
 'keymap': {'id': 'id',
  'link': 'id',
  'published_parsed': 'published_parsed',
  'summary': 'summary',
  'title': 'title'}}

In [100]:
import scrape_feeds

In [101]:
contents = scrape_feeds.get_feed_contents(rss_entry)

In [102]:
from bs4 import BeautifulSoup as bs

In [103]:
for c in contents:
    flag = False
    if len(c['title']) > 200:
        flag = True
    if len(c['link']) > 200:
        flag = True
    if len(c['summary']) > 5000:
        flag = True
    if flag:
        print(c)

In [106]:
for i in range(len(contents)):
    print(contents[i]['summary'])
    print('\n')

 Advocates who want to see America's immigration laws enforced have found a home in President Donald Trump's administration, the New York Times' Nicholas Kulish reports.


 Harmeet Dhillon, a San Francisco lawyer representing the UCB College Republicans, held a press conference Monday to discuss the case being brought against UC Berkeley and suggested deploying the National Guard to provide security if Berkeley's mayor could not maintain control of the city.


 Less than 100 days since leaving office, the former president has already arranged for a big Wall Street payday.


 Outsourcing firms, which supply major U.S. companies with thousands of foreign workers, are being called-out by the Trump White House.


 Actress Rachel Bloom has teamed up with "Science Guy" Bill Nye to debut an LGBT sex anthem on the new Netflix series 'Bill Nye Saves the World.'


 On the Tuesday edition of Breitbart News Daily, broadcast live on SiriusXM Patriot Channel 125 from 6AM to 9AM Eastern, Breitbart ed

    Only a single article.  No longer on the feed page.

### URL Errors

In [107]:
urle = errors[errors.err_class=="<class 'urllib.error.URLError'>"]
urle.shape

(1301, 5)

In [108]:
urle.tail()

,entry,day,entry_ind,is_error,err_class
2017-04-24 16:02:09,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:10,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:11,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:12,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:13,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>


In [109]:
urle.entry.unique()[:10]

array([ "Feed http://feeds.bbci.co.uk/news/world/latin_america/rss.xml: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed http://feeds.bbci.co.uk/news/world/middle_east/rss.xml: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed http://feeds.bbci.co.uk/news/world/us_and_canada/rss.xml: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed http://feeds.bbci.co.uk/news/video_and_audio/news_front_page/rss.xml?edition=uk: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed http://feeds.bbci.co.uk/news/video_and_audio/world/rss.xml: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed https://phys.org/rss-feed/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed https://phys.org/rss-feed/editorials/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed https://phys.org/rss-feed/earth-news/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)",
       "Feed

In [110]:
roots = urle.entry.apply(lambda x: x[5:22])

In [111]:
len(roots.unique())

2

In [116]:
for e in urle.entry.unique():
    print(e)
    print('\n')

Feed http://feeds.bbci.co.uk/news/world/latin_america/rss.xml: <class 'urllib.error.URLError'>: (GeneralProxyError(),)


Feed http://feeds.bbci.co.uk/news/world/middle_east/rss.xml: <class 'urllib.error.URLError'>: (GeneralProxyError(),)


Feed http://feeds.bbci.co.uk/news/world/us_and_canada/rss.xml: <class 'urllib.error.URLError'>: (GeneralProxyError(),)


Feed http://feeds.bbci.co.uk/news/video_and_audio/news_front_page/rss.xml?edition=uk: <class 'urllib.error.URLError'>: (GeneralProxyError(),)


Feed http://feeds.bbci.co.uk/news/video_and_audio/world/rss.xml: <class 'urllib.error.URLError'>: (GeneralProxyError(),)


Feed https://phys.org/rss-feed/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)


Feed https://phys.org/rss-feed/editorials/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)


Feed https://phys.org/rss-feed/earth-news/: <class 'urllib.error.URLError'>: (GeneralProxyError(),)


Feed https://phys.org/rss-feed/earth-news/earth-sciences/: <class 'urllib.err

In [112]:
feed_data = utils.load_feedlist_data('physorg_feedlist.xml')

In [113]:
for i,f in enumerate(feed_data[:15]):
    print(str(i) + ' ' + f['Link'])

0 https://phys.org/rss-feed/
1 https://phys.org/rss-feed/editorials/
2 https://phys.org/rss-feed/earth-news/
3 https://phys.org/rss-feed/earth-news/earth-sciences/
4 https://phys.org/rss-feed/earth-news/environment/
5 https://phys.org/rss-feed/science-news/
6 https://phys.org/rss-feed/science-news/archaeology-fossils/
7 https://phys.org/rss-feed/science-news/economics-business/
8 https://phys.org/rss-feed/science-news/mathematics/
9 https://phys.org/rss-feed/science-news/sci-other/
10 https://phys.org/rss-feed/science-news/social-sciences/
11 https://phys.org/rss-feed/nanotech-news/
12 https://phys.org/rss-feed/nanotech-news/bio-medicine/
13 https://phys.org/rss-feed/nanotech-news/nano-materials/
14 https://phys.org/rss-feed/nanotech-news/nano-physics/


In [115]:
urle.tail(10)

,entry,day,entry_ind,is_error,err_class
2017-04-24 16:02:04,Feed https://phys.org/rss-feed/biology-news/ec...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:05,Feed https://phys.org/rss-feed/biology-news/ev...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:06,Feed https://phys.org/rss-feed/biology-news/bi...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:07,Feed https://phys.org/rss-feed/biology-news/pl...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:08,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:09,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:10,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:11,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:12,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>
2017-04-24 16:02:13,Feed https://phys.org/rss-feed/chemistry-news/...,2017-04-24,388,True,<class 'urllib.error.URLError'>


    No issues since beginning of March; this issue seems to be fixed now.

### HTTP Errors

In [117]:
htpe = errors['2017-03-02 00:00:00':].copy()
htpe = htpe[htpe.err_class == "<class 'urllib.error.HTTPError'>"]
htpe.shape

(140, 5)

In [118]:
htpe.tail()

,entry,day,entry_ind,is_error,err_class
2017-04-21 08:03:06,Feed https://phys.org/rss-feed/biology-news/: ...,2017-04-21,367,True,<class 'urllib.error.HTTPError'>
2017-04-21 08:03:08,Feed https://phys.org/rss-feed/biology-news/bi...,2017-04-21,367,True,<class 'urllib.error.HTTPError'>
2017-04-21 11:03:23,Feed https://phys.org/rss-feed/biology-news/mi...,2017-04-21,370,True,<class 'urllib.error.HTTPError'>
2017-04-21 11:03:25,Feed https://phys.org/rss-feed/biology-news/ec...,2017-04-21,370,True,<class 'urllib.error.HTTPError'>
2017-04-21 11:03:28,Feed https://phys.org/rss-feed/biology-news/ev...,2017-04-21,370,True,<class 'urllib.error.HTTPError'>


In [119]:
roots = htpe.entry.apply(lambda x: x[5:22])

In [120]:
roots.unique()

array(['http://www.ft.com', 'http://feeds.foxn', 'http://rss.cnn.co',
       'http://feeds.feed', 'http://blogs.ft.c', 'https://www.thegu',
       'https://phys.org/', 'http://feeds.cnev'], dtype=object)

In [121]:
len(htpe.entry.unique())

48

In [122]:
htpe.entry.unique()

array([ "Feed http://www.ft.com/rss/world/mideast/economy: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://www.ft.com/rss/management/connected-business: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://www.ft.com/rss/brussels: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://www.ft.com/rss/world/uk/business: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://www.ft.com/rss/markets/asiapacific: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://www.ft.com/rss/companies/middleeast: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://www.ft.com/rss/world/uk/economy: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://www.ft.com/rss/world/mideast/politics: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://www.ft.com/rss/world/us/economy: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://feeds.foxnews.com/foxnews/latest: <class 'urllib.error.HTTPError'>: ()",
       "Feed http://feeds.foxnews.com/foxnews/most-pop

    This is strange.  Links seem to work.  May be a tor thing?  Let's see if this continues with change in settings for main...?

#### Incomplete Read Error

In [123]:
htpce = errors['2017-03-14 00:00:00':].copy()
htpce = htpce[htpce.err_class == "<class 'http.client.IncompleteRead'>"]
htpce.shape

(2, 5)

In [124]:
htpce

,entry,day,entry_ind,is_error,err_class
2017-03-22 17:11:10,Feed http://feeds.foxnews.com/foxnews/latest: ...,2017-03-22,90,True,<class 'http.client.IncompleteRead'>
2017-04-06 07:25:08,Feed http://feeds.cnevids.com/brand/wired.mrss...,2017-04-06,223,True,<class 'http.client.IncompleteRead'>


Not a big deal...